In [1]:
import random
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

/home/interaction-lab/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/interaction-lab/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# P3

In [2]:
df1 = pd.read_csv('p3_iser_data.csv')

In [3]:
df1.head()

,Timestamp,engagement,participant,session_num,session_date,activity,difficulty,total_games,session_games,total_mistakes,...,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,0.000000,engaged,3.0,17.0,2017-11-06,NaN,NaN,1,1,0,...,1.0,0.26013400000000003,0.25702600000000003,-0.930735,-0.07774099999999999,0.3718010000000001,-0.925052,0.098,0.327,1.0
1,0.033333,engaged,3.0,17.0,2017-11-06,NaN,NaN,1,1,0,...,1.0,0.26363000000000003,0.355734,-0.896634,-0.022728,0.415454,-0.90933,0.133,0.404,1.0
2,0.066667,engaged,3.0,17.0,2017-11-06,NaN,NaN,1,1,0,...,1.0,0.265639,0.282356,-0.921798,-0.049192,0.379725,-0.92379,0.117,0.344,1.0
3,0.100000,engaged,3.0,17.0,2017-11-06,NaN,NaN,1,1,0,...,1.0,0.245775,0.366982,-0.897172,-0.043081,0.407407,-0.91223,0.112,0.404,1.0
4,0.133333,engaged,3.0,17.0,2017-11-06,NaN,NaN,1,1,0,...,1.0,0.209906,0.365945,-0.906655,-0.06716,0.417929,-0.905994,0.079,0.408,1.0


In [4]:
df1.isna().sum()

Timestamp                     0
engagement                   10
participant                   0
session_num                   0
session_date                  0
activity                  31588
difficulty                31588
total_games                   0
session_games                 0
total_mistakes                0
session_mistakes              0
game_mistakes                 0
ros_ROBOT_STATE           84544
ros_PARTICIPANT_STATE    128559
of_timestamp                128
of_confidence               128
of_success                  128
of_gaze_0_x                 128
of_gaze_0_y                 128
of_gaze_0_z                 128
of_gaze_1_x                 128
of_gaze_1_y                 128
of_gaze_1_z                 128
of_gaze_angle_x             128
of_gaze_angle_y             128
op_Number of People         112
dtype: int64

In [5]:
df1 = df1.drop(['ros_ROBOT_STATE','ros_PARTICIPANT_STATE','session_date','Timestamp'],axis=1)

In [6]:
df1.isna().sum()

engagement                10
participant                0
session_num                0
activity               31588
difficulty             31588
total_games                0
session_games              0
total_mistakes             0
session_mistakes           0
game_mistakes              0
of_timestamp             128
of_confidence            128
of_success               128
of_gaze_0_x              128
of_gaze_0_y              128
of_gaze_0_z              128
of_gaze_1_x              128
of_gaze_1_y              128
of_gaze_1_z              128
of_gaze_angle_x          128
of_gaze_angle_y          128
op_Number of People      112
dtype: int64

In [7]:
df1 = df1.fillna(value = {'activity':-1,'difficulty':-1 })

In [8]:
df1.head()

,engagement,participant,session_num,activity,difficulty,total_games,session_games,total_mistakes,session_mistakes,game_mistakes,...,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.26013400000000003,0.25702600000000003,-0.930735,-0.07774099999999999,0.3718010000000001,-0.925052,0.098,0.327,1.0
1,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.26363000000000003,0.355734,-0.896634,-0.022728,0.415454,-0.90933,0.133,0.404,1.0
2,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.265639,0.282356,-0.921798,-0.049192,0.379725,-0.92379,0.117,0.344,1.0
3,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.245775,0.366982,-0.897172,-0.043081,0.407407,-0.91223,0.112,0.404,1.0
4,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.209906,0.365945,-0.906655,-0.06716,0.417929,-0.905994,0.079,0.408,1.0


In [9]:
df1.isna().sum()

engagement              10
participant              0
session_num              0
activity                 0
difficulty               0
total_games              0
session_games            0
total_mistakes           0
session_mistakes         0
game_mistakes            0
of_timestamp           128
of_confidence          128
of_success             128
of_gaze_0_x            128
of_gaze_0_y            128
of_gaze_0_z            128
of_gaze_1_x            128
of_gaze_1_y            128
of_gaze_1_z            128
of_gaze_angle_x        128
of_gaze_angle_y        128
op_Number of People    112
dtype: int64

In [10]:
df1 = df1.dropna()
#df1.to_csv('p3.csv',index=False)

In [11]:
df1.head()

,engagement,participant,session_num,activity,difficulty,total_games,session_games,total_mistakes,session_mistakes,game_mistakes,...,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.26013400000000003,0.25702600000000003,-0.930735,-0.07774099999999999,0.3718010000000001,-0.925052,0.098,0.327,1.0
1,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.26363000000000003,0.355734,-0.896634,-0.022728,0.415454,-0.90933,0.133,0.404,1.0
2,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.265639,0.282356,-0.921798,-0.049192,0.379725,-0.92379,0.117,0.344,1.0
3,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.245775,0.366982,-0.897172,-0.043081,0.407407,-0.91223,0.112,0.404,1.0
4,engaged,3.0,17.0,-1.0,-1.0,1,1,0,0,0,...,1.0,0.209906,0.365945,-0.906655,-0.06716,0.417929,-0.905994,0.079,0.408,1.0


In [12]:
len(df1)

139850

In [13]:
df1 = df1[df1.of_gaze_0_y != ' nan']

In [14]:
len(df1)
#df1.to_csv('p3.csv',index=False)

125286

In [15]:
df1 = df1.drop(['participant','session_num'],axis=1)

In [16]:
df1.head()

,engagement,activity,difficulty,total_games,session_games,total_mistakes,session_mistakes,game_mistakes,of_timestamp,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,engaged,-1.0,-1.0,1,1,0,0,0,0.000,0.88,1.0,0.26013400000000003,0.25702600000000003,-0.930735,-0.07774099999999999,0.3718010000000001,-0.925052,0.098,0.327,1.0
1,engaged,-1.0,-1.0,1,1,0,0,0,0.033,0.98,1.0,0.26363000000000003,0.355734,-0.896634,-0.022728,0.415454,-0.90933,0.133,0.404,1.0
2,engaged,-1.0,-1.0,1,1,0,0,0,0.067,0.98,1.0,0.265639,0.282356,-0.921798,-0.049192,0.379725,-0.92379,0.117,0.344,1.0
3,engaged,-1.0,-1.0,1,1,0,0,0,0.100,0.98,1.0,0.245775,0.366982,-0.897172,-0.043081,0.407407,-0.91223,0.112,0.404,1.0
4,engaged,-1.0,-1.0,1,1,0,0,0,0.133,0.98,1.0,0.209906,0.365945,-0.906655,-0.06716,0.417929,-0.905994,0.079,0.408,1.0


In [17]:
df1['engagement'].value_counts()

engaged       111378
disengaged     13908
Name: engagement, dtype: int64

In [18]:
print("Engaged",df1['engagement'].value_counts()['engaged']/sum(df1['engagement'].value_counts()))
print("Disengaged",df1['engagement'].value_counts()['disengaged']/sum(df1['engagement'].value_counts()))

Engaged 0.8889899909008189
Disengaged 0.11101000909918107


In [19]:
le = LabelEncoder()
df1['engagement'] = le.fit_transform(df1['engagement'])

In [20]:
df1.head()

,engagement,activity,difficulty,total_games,session_games,total_mistakes,session_mistakes,game_mistakes,of_timestamp,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,1,-1.0,-1.0,1,1,0,0,0,0.000,0.88,1.0,0.26013400000000003,0.25702600000000003,-0.930735,-0.07774099999999999,0.3718010000000001,-0.925052,0.098,0.327,1.0
1,1,-1.0,-1.0,1,1,0,0,0,0.033,0.98,1.0,0.26363000000000003,0.355734,-0.896634,-0.022728,0.415454,-0.90933,0.133,0.404,1.0
2,1,-1.0,-1.0,1,1,0,0,0,0.067,0.98,1.0,0.265639,0.282356,-0.921798,-0.049192,0.379725,-0.92379,0.117,0.344,1.0
3,1,-1.0,-1.0,1,1,0,0,0,0.100,0.98,1.0,0.245775,0.366982,-0.897172,-0.043081,0.407407,-0.91223,0.112,0.404,1.0
4,1,-1.0,-1.0,1,1,0,0,0,0.133,0.98,1.0,0.209906,0.365945,-0.906655,-0.06716,0.417929,-0.905994,0.079,0.408,1.0


## Linear Kernel SVM

### Test: 20% - Random

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df1.iloc[:,1:], df1.iloc[:,0], test_size=0.2, random_state=42)

In [22]:
clf = LinearSVC()

In [23]:
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [24]:
pred = clf.predict(X_test)
scores = clf.decision_function(X_test)

In [25]:
print("AUC:",roc_auc_score(y_test, scores)) 

AUC: 0.6486868368252813


In [26]:
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

Accuracy: 0.8911724798467555
[[   70  2688]
 [   39 22261]]


In [27]:
print(metrics.classification_report(y_test, pred))

             precision    recall  f1-score   support

          0       0.64      0.03      0.05      2758
          1       0.89      1.00      0.94     22300

avg / total       0.86      0.89      0.84     25058



### Test: 30% - Last by session

In [50]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df1.iloc[:,1:], df1.iloc[:,0], test_size=0.3, shuffle=False)
clf1 = LinearSVC()
clf1.fit(X_train1,y_train1)
pred1 = clf1.predict(X_test1)
scores1 = clf1.decision_function(X_test1)

In [51]:
print("AUC:",roc_auc_score(y_test1, scores1)) 
print("Accuracy:",accuracy_score(y_test1, pred1))
print(metrics.confusion_matrix(y_test1, pred1))
print(metrics.classification_report(y_test1, pred1))

AUC: 0.6761621911113234
Accuracy: 0.8387431490448571
[[  327  5346]
 [  715 31198]]
             precision    recall  f1-score   support

          0       0.31      0.06      0.10      5673
          1       0.85      0.98      0.91     31913

avg / total       0.77      0.84      0.79     37586



### Cross Validation

### 5 - fold CV

In [67]:
X = df1.iloc[:,1:]
y = df1.iloc[:,0]
clf11 = LinearSVC()
y_pred = cross_val_predict(clf11, X, y, cv=5)

In [68]:
print("Accuracy:",accuracy_score(y, y_pred))
print(metrics.confusion_matrix(y, y_pred))
print(metrics.classification_report(y, y_pred))

Accuracy: 0.790487364909088
[[ 3123 10785]
 [15464 95914]]
             precision    recall  f1-score   support

          0       0.17      0.22      0.19     13908
          1       0.90      0.86      0.88    111378

avg / total       0.82      0.79      0.80    125286



### 3 - fold CV

In [72]:
scores = cross_val_score(clf11, X, y, cv=3)

In [73]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.80 (+/- 0.22)


### Feature Selection

In [36]:
X_train11, X_test11, y_train11, y_test11 = train_test_split(df1.iloc[:,1:], df1.iloc[:,0], test_size=0.2, random_state=42)

In [42]:
clf11 = LinearSVC()
selector = RFE(clf11, 5)
selector = selector.fit(X_train11,y_train11)
pred11 = selector.predict(X_test11)
scores11 = selector.decision_function(X_test11)

In [43]:
print("AUC:",roc_auc_score(y_test11, scores11)) 
print("Accuracy:",accuracy_score(y_test11, pred11))
print(metrics.confusion_matrix(y_test11, pred11))
print(metrics.classification_report(y_test11, pred11))

AUC: 0.7855252555143293
Accuracy: 0.8906137760395881
[[   69  2689]
 [   52 22248]]
             precision    recall  f1-score   support

          0       0.57      0.03      0.05      2758
          1       0.89      1.00      0.94     22300

avg / total       0.86      0.89      0.84     25058



In [44]:
selector.get_support()

array([False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False,  True,  True, False,  True,
       False])

In [46]:
selector.ranking_ 

array([ 9, 11, 14, 12, 13, 10,  8, 15,  1,  7,  2,  1,  4,  3,  1,  1,  5,
        1,  6])

In [47]:
list(df1)

['engagement',
 'activity',
 'difficulty',
 'total_games',
 'session_games',
 'total_mistakes',
 'session_mistakes',
 'game_mistakes',
 'of_timestamp',
 'of_confidence',
 'of_success',
 'of_gaze_0_x',
 'of_gaze_0_y',
 'of_gaze_0_z',
 'of_gaze_1_x',
 'of_gaze_1_y',
 'of_gaze_1_z',
 'of_gaze_angle_x',
 'of_gaze_angle_y',
 'op_Number of People']

In [59]:
for i,j in zip(selector.ranking_,list(df1)):
    if(i==1):
        print(j)

of_timestamp
of_gaze_0_x
of_gaze_1_x
of_gaze_1_y
of_gaze_angle_x


In [57]:
for i,j in zip(selector.ranking_,list(df1)):
    print(j,i)

engagement 9
activity 11
difficulty 14
total_games 12
session_games 13
total_mistakes 10
session_mistakes 8
game_mistakes 15
of_timestamp 1
of_confidence 7
of_success 2
of_gaze_0_x 1
of_gaze_0_y 4
of_gaze_0_z 3
of_gaze_1_x 1
of_gaze_1_y 1
of_gaze_1_z 5
of_gaze_angle_x 1
of_gaze_angle_y 6


### Sampling

#### Oversampling

In [65]:
X = df1.iloc[:,1:]
y = df1.iloc[:,0]
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

In [68]:
print(sorted(Counter(y_resampled).items()))

[(0, 111378), (1, 111378)]


In [69]:
X_train12, X_test12, y_train12, y_test12 = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
clf12 = LinearSVC()
clf12.fit(X_train12,y_train12)
pred12 = clf12.predict(X_test12)
scores12 = clf12.decision_function(X_test12)

In [70]:
print("AUC:",roc_auc_score(y_test12, scores12)) 
print("Accuracy:",accuracy_score(y_test12, pred12))
print(metrics.confusion_matrix(y_test12, pred12))
print(metrics.classification_report(y_test12, pred12))

AUC: 0.7494637333409434
Accuracy: 0.6379960495600646
[[ 8614 13707]
 [ 2421 19810]]
             precision    recall  f1-score   support

          0       0.78      0.39      0.52     22321
          1       0.59      0.89      0.71     22231

avg / total       0.69      0.64      0.61     44552



#### Undersampling

### Standarization of data

##### Standard Scalar

In [24]:
X = df1.iloc[:,1:]
y = df1.iloc[:,0]
sscaler = StandardScaler()
Standard_X = sscaler.fit_transform(X)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [76]:
X_train13, X_test13, y_train13, y_test13 = train_test_split(Standard_X, y, test_size=0.2, random_state=42)
clf13 = LinearSVC()
clf13.fit(X_train13,y_train13)
pred13 = clf13.predict(X_test13)
scores13 = clf13.decision_function(X_test13)

In [77]:
print("AUC:",roc_auc_score(y_test13, scores13)) 
print("Accuracy:",accuracy_score(y_test13, pred13))
print(metrics.confusion_matrix(y_test13, pred13))
print(metrics.classification_report(y_test13, pred13))

AUC: 0.8250178526715597
Accuracy: 0.8898156277436348
[[  186  2572]
 [  189 22111]]
             precision    recall  f1-score   support

          0       0.50      0.07      0.12      2758
          1       0.90      0.99      0.94     22300

avg / total       0.85      0.89      0.85     25058



##### Robust Scalar

In [25]:
rscaler = RobustScaler()
Robust_X = rscaler.fit_transform(X)

In [79]:
X_train13, X_test13, y_train13, y_test13 = train_test_split(Robust_X, y, test_size=0.2, random_state=42)
clf13 = LinearSVC()
clf13.fit(X_train13,y_train13)
pred13 = clf13.predict(X_test13)
scores13 = clf13.decision_function(X_test13)

In [80]:
print("AUC:",roc_auc_score(y_test13, scores13)) 
print("Accuracy:",accuracy_score(y_test13, pred13))
print(metrics.confusion_matrix(y_test13, pred13))
print(metrics.classification_report(y_test13, pred13))

AUC: 0.8250158202635953
Accuracy: 0.8898555351584324
[[  186  2572]
 [  188 22112]]
             precision    recall  f1-score   support

          0       0.50      0.07      0.12      2758
          1       0.90      0.99      0.94     22300

avg / total       0.85      0.89      0.85     25058



## SVM - Poly / RBF Kernel

In [26]:
X_train13, X_test13, y_train13, y_test13 = train_test_split(Robust_X, y, test_size=0.2, random_state=42)
clf14 = svm.SVC(kernel='poly', degree=2)
clf14.fit(X_train13,y_train13)
pred14 = clf14.predict(X_test13)
scores14 = clf14.decision_function(X_test13)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [27]:
print("AUC:",roc_auc_score(y_test13, scores14)) 
print("Accuracy:",accuracy_score(y_test13, pred14))
print(metrics.confusion_matrix(y_test13, pred14))
print(metrics.classification_report(y_test13, pred14))

AUC: 0.8850225190802461
Accuracy: 0.9100885944608508
[[  640  2118]
 [  135 22165]]
              precision    recall  f1-score   support

           0       0.83      0.23      0.36      2758
           1       0.91      0.99      0.95     22300

   micro avg       0.91      0.91      0.91     25058
   macro avg       0.87      0.61      0.66     25058
weighted avg       0.90      0.91      0.89     25058



In [ ]:
#Degree = 3

In [28]:
clf15 = svm.SVC(kernel='poly', degree=3)
clf15.fit(X_train13,y_train13)
pred15 = clf15.predict(X_test13)
scores15 = clf15.decision_function(X_test13)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [29]:
print("AUC:",roc_auc_score(y_test13, scores15)) 
print("Accuracy:",accuracy_score(y_test13, pred15))
print(metrics.confusion_matrix(y_test13, pred15))
print(metrics.classification_report(y_test13, pred15))

AUC: 0.9161942429199035
Accuracy: 0.927448319897837
[[ 1142  1616]
 [  202 22098]]
              precision    recall  f1-score   support

           0       0.85      0.41      0.56      2758
           1       0.93      0.99      0.96     22300

   micro avg       0.93      0.93      0.93     25058
   macro avg       0.89      0.70      0.76     25058
weighted avg       0.92      0.93      0.92     25058



In [ ]:
# RBF Kernel

In [30]:
clf16 = svm.SVC(kernel='rbf')
clf16.fit(X_train13,y_train13)
pred16 = clf16.predict(X_test13)
scores16 = clf16.decision_function(X_test13)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [31]:
print("AUC:",roc_auc_score(y_test13, scores16)) 
print("Accuracy:",accuracy_score(y_test13, pred16))
print(metrics.confusion_matrix(y_test13, pred16))
print(metrics.classification_report(y_test13, pred16))

AUC: 0.9498453581428018
Accuracy: 0.9390613776039588
[[ 1412  1346]
 [  181 22119]]
              precision    recall  f1-score   support

           0       0.89      0.51      0.65      2758
           1       0.94      0.99      0.97     22300

   micro avg       0.94      0.94      0.94     25058
   macro avg       0.91      0.75      0.81     25058
weighted avg       0.94      0.94      0.93     25058



# P5

In [21]:
df2 = pd.read_csv('p5_iser_data.csv')
df2.head()

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Timestamp,engagement,participant,session_num,session_date,activity,difficulty,total_games,session_games,total_mistakes,...,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,0.000000,engaged,5.0,1.0,2018-04-05,NaN,NaN,0,0,0,...,1.0,0.076070,0.127410,-0.988929,-0.294418,0.170766,-0.940296,-0.113,0.153,2.0
1,0.033333,engaged,5.0,1.0,2018-04-05,NaN,NaN,0,0,0,...,1.0,0.100135,0.130810,-0.986337,-0.277336,0.177454,-0.944243,-0.092,0.158,2.0
2,0.066667,engaged,5.0,1.0,2018-04-05,NaN,NaN,0,0,0,...,1.0,0.087110,0.121171,-0.988802,-0.325118,0.173623,-0.929599,-0.123,0.152,2.0
3,0.100000,engaged,5.0,1.0,2018-04-05,NaN,NaN,0,0,0,...,1.0,0.089518,0.094099,-0.991530,-0.322802,0.183486,-0.928510,-0.121,0.144,2.0
4,0.133333,engaged,5.0,1.0,2018-04-05,NaN,NaN,0,0,0,...,1.0,0.096125,0.090759,-0.991223,-0.322998,0.173539,-0.930353,-0.118,0.137,2.0


In [22]:
df2.isna().sum()

Timestamp                     0
engagement                    0
participant                   0
session_num                   0
session_date                  0
activity                  96630
difficulty                96630
total_games                   0
session_games                 0
total_mistakes                0
session_mistakes              0
game_mistakes                 0
ros_ROBOT_STATE          265034
ros_PARTICIPANT_STATE    320726
of_timestamp                  0
of_confidence                 0
of_success                    0
of_gaze_0_x                   0
of_gaze_0_y                   0
of_gaze_0_z                   0
of_gaze_1_x                   0
of_gaze_1_y                   0
of_gaze_1_z                   0
of_gaze_angle_x               0
of_gaze_angle_y               0
op_Number of People           0
dtype: int64

In [23]:
df2 = df2.drop(['ros_ROBOT_STATE','ros_PARTICIPANT_STATE','session_date','Timestamp'],axis=1)
df2 = df2.fillna(value = {'activity':-1,'difficulty':-1 })
df2 = df2.dropna()

In [24]:
df2.head()

,engagement,participant,session_num,activity,difficulty,total_games,session_games,total_mistakes,session_mistakes,game_mistakes,...,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,engaged,5.0,1.0,-1.0,-1.0,0,0,0,0,0,...,1.0,0.076070,0.127410,-0.988929,-0.294418,0.170766,-0.940296,-0.113,0.153,2.0
1,engaged,5.0,1.0,-1.0,-1.0,0,0,0,0,0,...,1.0,0.100135,0.130810,-0.986337,-0.277336,0.177454,-0.944243,-0.092,0.158,2.0
2,engaged,5.0,1.0,-1.0,-1.0,0,0,0,0,0,...,1.0,0.087110,0.121171,-0.988802,-0.325118,0.173623,-0.929599,-0.123,0.152,2.0
3,engaged,5.0,1.0,-1.0,-1.0,0,0,0,0,0,...,1.0,0.089518,0.094099,-0.991530,-0.322802,0.183486,-0.928510,-0.121,0.144,2.0
4,engaged,5.0,1.0,-1.0,-1.0,0,0,0,0,0,...,1.0,0.096125,0.090759,-0.991223,-0.322998,0.173539,-0.930353,-0.118,0.137,2.0


In [25]:
len(df2)

339128

In [25]:
#df2.to_csv('p5.csv')

In [26]:
df2 = df2.drop(['participant','session_num'],axis=1)

In [27]:
df2['engagement'].value_counts()

engaged       232553
disengaged    106575
Name: engagement, dtype: int64

In [28]:
df2['engagement'].value_counts()
print("Engaged",df2['engagement'].value_counts()['engaged']/sum(df2['engagement'].value_counts()))
print("Disengaged",df2['engagement'].value_counts()['disengaged']/sum(df2['engagement'].value_counts()))

Engaged 0.6857381283763063
Disengaged 0.31426187162369373


In [29]:
le2 = LabelEncoder()
df2['engagement'] = le2.fit_transform(df2['engagement'])

In [30]:
df2.head()

,engagement,activity,difficulty,total_games,session_games,total_mistakes,session_mistakes,game_mistakes,of_timestamp,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,1,-1.0,-1.0,0,0,0,0,0,0.000,0.98,1.0,0.076070,0.127410,-0.988929,-0.294418,0.170766,-0.940296,-0.113,0.153,2.0
1,1,-1.0,-1.0,0,0,0,0,0,0.033,0.98,1.0,0.100135,0.130810,-0.986337,-0.277336,0.177454,-0.944243,-0.092,0.158,2.0
2,1,-1.0,-1.0,0,0,0,0,0,0.067,0.98,1.0,0.087110,0.121171,-0.988802,-0.325118,0.173623,-0.929599,-0.123,0.152,2.0
3,1,-1.0,-1.0,0,0,0,0,0,0.100,0.98,1.0,0.089518,0.094099,-0.991530,-0.322802,0.183486,-0.928510,-0.121,0.144,2.0
4,1,-1.0,-1.0,0,0,0,0,0,0.133,0.98,1.0,0.096125,0.090759,-0.991223,-0.322998,0.173539,-0.930353,-0.118,0.137,2.0


## Linear Kernel SVM

### Test: 20% - Random

In [48]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df2.iloc[:,1:], df2.iloc[:,0], test_size=0.2, random_state=42)
clf2 = LinearSVC()
clf2.fit(X_train2,y_train2)
pred2 = clf2.predict(X_test2)
scores2 = clf2.decision_function(X_test2)

In [49]:
print("AUC:",roc_auc_score(y_test2, scores2)) 
print("Accuracy:",accuracy_score(y_test2, pred2))
print(metrics.confusion_matrix(y_test2, pred2))
print(metrics.classification_report(y_test2, pred2))

AUC: 0.9047571768391618
Accuracy: 0.8493498068587267
[[11421 10096]
 [  122 46187]]
             precision    recall  f1-score   support

          0       0.99      0.53      0.69     21517
          1       0.82      1.00      0.90     46309

avg / total       0.87      0.85      0.83     67826



### Test: 30% - Random

In [60]:
X_train21, X_test21, y_train21, y_test21 = train_test_split(df2.iloc[:,1:], df2.iloc[:,0], test_size=0.3)
clf21 = LinearSVC()
clf21.fit(X_train21,y_train21)
pred21 = clf21.predict(X_test21)
scores21 = clf21.decision_function(X_test21)

In [61]:
print("AUC:",roc_auc_score(y_test21, scores21)) 
print("Accuracy:",accuracy_score(y_test21, pred21))
print(metrics.confusion_matrix(y_test21, pred21))
print(metrics.classification_report(y_test21, pred21))

AUC: 0.9084512027330167
Accuracy: 0.8732934272992657
[[21934 10150]
 [ 2741 66914]]
             precision    recall  f1-score   support

          0       0.89      0.68      0.77     32084
          1       0.87      0.96      0.91     69655

avg / total       0.87      0.87      0.87    101739



### Test: 30% - Last by session

In [62]:
X_train21, X_test21, y_train21, y_test21 = train_test_split(df2.iloc[:,1:], df2.iloc[:,0], test_size=0.3, shuffle=False)
clf21 = LinearSVC()
clf21.fit(X_train21,y_train21)
pred21 = clf21.predict(X_test21)
scores21 = clf21.decision_function(X_test21)

In [63]:
print("AUC:",roc_auc_score(y_test21, scores21)) 
print("Accuracy:",accuracy_score(y_test21, pred21))
print(metrics.confusion_matrix(y_test21, pred21))
print(metrics.classification_report(y_test21, pred21))

AUC: 0.6626892813103046
Accuracy: 0.8046865017348312
[[ 2426 18773]
 [ 1098 79442]]
             precision    recall  f1-score   support

          0       0.69      0.11      0.20     21199
          1       0.81      0.99      0.89     80540

avg / total       0.78      0.80      0.74    101739



### Cross Validation

### 5 - fold CV

In [42]:
X2 = df2.iloc[:,1:]
y2 = df2.iloc[:,0]
clf21 = LinearSVC()
y_pred2 = cross_val_predict(clf21, X2, y2, cv=5)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblin

In [43]:
print("Accuracy:",accuracy_score(y2, y_pred2))
print(metrics.confusion_matrix(y2, y_pred2))
print(metrics.classification_report(y2, y_pred2))

Accuracy: 0.8006121582411361
[[ 75931  30644]
 [ 36974 195579]]
              precision    recall  f1-score   support

           0       0.67      0.71      0.69    106575
           1       0.86      0.84      0.85    232553

   micro avg       0.80      0.80      0.80    339128
   macro avg       0.77      0.78      0.77    339128
weighted avg       0.80      0.80      0.80    339128



### 3 - fold CV

In [44]:
scores = cross_val_score(clf21, X2, y2, cv=3)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [45]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.83 (+/- 0.23)


### Feature Selection

In [46]:
X_train21, X_test21, y_train21, y_test21 = train_test_split(df2.iloc[:,1:], df2.iloc[:,0], test_size=0.2, random_state=42)

In [52]:
clf21 = LinearSVC()
selector = RFE(clf21, 5)
selector = selector.fit(X_train21,y_train21)
pred21 = selector.predict(X_test21)
scores21 = selector.decision_function(X_test21)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblin

In [50]:
#scores21 = selector.decision_function(X_test21)

In [53]:
print("AUC:",roc_auc_score(y_test21, scores21)) 
print("Accuracy:",accuracy_score(y_test21, pred21))
print(metrics.confusion_matrix(y_test21, pred21))
print(metrics.classification_report(y_test21, pred21))

AUC: 0.8538208053480261
Accuracy: 0.8289004216672072
[[16380  5137]
 [ 6468 39841]]
              precision    recall  f1-score   support

           0       0.72      0.76      0.74     21517
           1       0.89      0.86      0.87     46309

   micro avg       0.83      0.83      0.83     67826
   macro avg       0.80      0.81      0.81     67826
weighted avg       0.83      0.83      0.83     67826



In [54]:
selector.get_support()

array([False, False, False, False, False, False, False, False,  True,
        True,  True, False, False,  True, False,  True, False, False,
       False])

In [55]:
selector.ranking_ 

array([10,  8, 14, 13, 12, 11,  9, 15,  1,  1,  1,  7,  4,  1,  3,  1,  6,
        2,  5])

In [56]:
list(df2)

['engagement',
 'activity',
 'difficulty',
 'total_games',
 'session_games',
 'total_mistakes',
 'session_mistakes',
 'game_mistakes',
 'of_timestamp',
 'of_confidence',
 'of_success',
 'of_gaze_0_x',
 'of_gaze_0_y',
 'of_gaze_0_z',
 'of_gaze_1_x',
 'of_gaze_1_y',
 'of_gaze_1_z',
 'of_gaze_angle_x',
 'of_gaze_angle_y',
 'op_Number of People']

In [57]:
for i,j in zip(selector.ranking_,list(df2)):
    if(i==1):
        print(j)

of_timestamp
of_confidence
of_success
of_gaze_0_z
of_gaze_1_y


In [58]:
for i,j in zip(selector.ranking_,list(df2)):
    print(j,i)

engagement 10
activity 8
difficulty 14
total_games 13
session_games 12
total_mistakes 11
session_mistakes 9
game_mistakes 15
of_timestamp 1
of_confidence 1
of_success 1
of_gaze_0_x 7
of_gaze_0_y 4
of_gaze_0_z 1
of_gaze_1_x 3
of_gaze_1_y 1
of_gaze_1_z 6
of_gaze_angle_x 2
of_gaze_angle_y 5


### Sampling

#### Oversampling

In [59]:
X2 = df2.iloc[:,1:]
y2 = df2.iloc[:,0]
X_resampled2, y_resampled2 = SMOTE().fit_resample(X2, y2)

In [60]:
print(sorted(Counter(y_resampled2).items()))

[(0, 232553), (1, 232553)]


In [61]:
X_train22, X_test22, y_train22, y_test22 = train_test_split(X_resampled2, y_resampled2, test_size=0.2, random_state=42)
clf22 = LinearSVC()
clf22.fit(X_train22,y_train22)
pred22 = clf22.predict(X_test22)
scores22 = clf22.decision_function(X_test22)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [62]:
print("AUC:",roc_auc_score(y_test22, scores22)) 
print("Accuracy:",accuracy_score(y_test22, pred22))
print(metrics.confusion_matrix(y_test22, pred22))
print(metrics.classification_report(y_test22, pred22))

AUC: 0.8695138700254192
Accuracy: 0.7772247425340242
[[29017 17589]
 [ 3134 43282]]
              precision    recall  f1-score   support

           0       0.90      0.62      0.74     46606
           1       0.71      0.93      0.81     46416

   micro avg       0.78      0.78      0.78     93022
   macro avg       0.81      0.78      0.77     93022
weighted avg       0.81      0.78      0.77     93022



#### Undersampling

### Standarization of data

##### Standard Scalar

In [32]:
X2 = df2.iloc[:,1:]
y2 = df2.iloc[:,0]
sscaler = StandardScaler()
Standard_X2 = sscaler.fit_transform(X2)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [65]:
X_train23, X_test23, y_train23, y_test23 = train_test_split(Standard_X2, y2, test_size=0.2, random_state=42)
clf23 = LinearSVC()
clf23.fit(X_train23,y_train23)
pred23 = clf23.predict(X_test23)
scores23 = clf23.decision_function(X_test23)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [66]:
print("AUC:",roc_auc_score(y_test23, scores23)) 
print("Accuracy:",accuracy_score(y_test23, pred23))
print(metrics.confusion_matrix(y_test23, pred23))
print(metrics.classification_report(y_test23, pred23))

AUC: 0.9427436700159735
Accuracy: 0.8756671482912157
[[15020  6497]
 [ 1936 44373]]
              precision    recall  f1-score   support

           0       0.89      0.70      0.78     21517
           1       0.87      0.96      0.91     46309

   micro avg       0.88      0.88      0.88     67826
   macro avg       0.88      0.83      0.85     67826
weighted avg       0.88      0.88      0.87     67826



##### Robust Scalar

In [33]:
rscaler = RobustScaler()
Robust_X2 = rscaler.fit_transform(X2)

In [34]:
X_train23, X_test23, y_train23, y_test23 = train_test_split(Robust_X2, y2, test_size=0.2, random_state=42)
clf23 = LinearSVC()
clf23.fit(X_train23,y_train23)
pred23 = clf23.predict(X_test23)
scores23 = clf23.decision_function(X_test23)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [35]:
print("AUC:",roc_auc_score(y_test23, scores23)) 
print("Accuracy:",accuracy_score(y_test23, pred23))
print(metrics.confusion_matrix(y_test23, pred23))
print(metrics.classification_report(y_test23, pred23))

AUC: 0.9427382095261364
Accuracy: 0.8756818918998615
[[15021  6496]
 [ 1936 44373]]
              precision    recall  f1-score   support

           0       0.89      0.70      0.78     21517
           1       0.87      0.96      0.91     46309

   micro avg       0.88      0.88      0.88     67826
   macro avg       0.88      0.83      0.85     67826
weighted avg       0.88      0.88      0.87     67826



## SVM - Poly / RBF Kernel

In [ ]:
#X_train23, X_test23, y_train23, y_test23 = train_test_split(Robust_X2, y2, test_size=0.2, random_state=42)
clf24 = svm.SVC(kernel='poly', degree=2)
clf24.fit(X_train23,y_train23)
pred24 = clf24.predict(X_test23)
scores24 = clf24.decision_function(X_test23)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
print("AUC:",roc_auc_score(y_test23, scores24)) 
print("Accuracy:",accuracy_score(y_test23, pred24))
print(metrics.confusion_matrix(y_test23, pred24))
print(metrics.classification_report(y_test23, pred24))

## P3 and P5 - Combined Test - 20% Random

In [31]:
frames = [df1, df2]
combined = pd.concat(frames)

In [39]:
print(combined['engagement'].value_counts())
print("Engaged",combined['engagement'].value_counts()[1]/sum(combined['engagement'].value_counts()))
print("Disengaged",combined['engagement'].value_counts()[0]/sum(combined['engagement'].value_counts()))

1    343931
0    120483
Name: engagement, dtype: int64
Engaged 0.7405698363959743
Disengaged 0.2594301636040257


In [40]:
X_trainc, X_testc, y_trainc, y_testc = train_test_split(combined.iloc[:,1:], combined.iloc[:,0], test_size=0.2, random_state=42)
clfc = LinearSVC()
clfc.fit(X_trainc,y_trainc)
predc = clfc.predict(X_testc)
scoresc = clfc.decision_function(X_testc)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [41]:
print("AUC:",roc_auc_score(y_testc, scoresc)) 
print("Accuracy:",accuracy_score(y_testc, predc))
print(metrics.confusion_matrix(y_testc, predc))
print(metrics.classification_report(y_testc, predc))

AUC: 0.8488516527612686
Accuracy: 0.7796475135385378
[[ 3627 20418]
 [   49 68789]]
              precision    recall  f1-score   support

           0       0.99      0.15      0.26     24045
           1       0.77      1.00      0.87     68838

   micro avg       0.78      0.78      0.78     92883
   macro avg       0.88      0.58      0.57     92883
weighted avg       0.83      0.78      0.71     92883



## Train - P3 & Test - P5

In [42]:
X_P3 = df1.iloc[:,1:]
y_P3 = df1.iloc[:,0]
X_P5 = df2.iloc[:,1:]
y_P5 = df2.iloc[:,0]

In [43]:
clfp35 = LinearSVC()
clfp35.fit(X_P3,y_P3)
predp35 = clfp35.predict(X_P5)
scoresp35 = clfp35.decision_function(X_P5)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [44]:
print("AUC:",roc_auc_score(y_P5, scoresp35)) 
print("Accuracy:",accuracy_score(y_P5, predp35))
print(metrics.confusion_matrix(y_P5, predp35))
print(metrics.classification_report(y_P5, predp35))

AUC: 0.7928635901652394
Accuracy: 0.673660092944257
[[ 11054  95521]
 [ 15150 217403]]
              precision    recall  f1-score   support

           0       0.42      0.10      0.17    106575
           1       0.69      0.93      0.80    232553

   micro avg       0.67      0.67      0.67    339128
   macro avg       0.56      0.52      0.48    339128
weighted avg       0.61      0.67      0.60    339128



## Train - P5 & Test - P3

In [45]:
clfp53 = LinearSVC()
clfp53.fit(X_P5,y_P5)
predp53 = clfp53.predict(X_P3)
scoresp53 = clfp53.decision_function(X_P3)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [46]:
print("AUC:",roc_auc_score(y_P3, scoresp53)) 
print("Accuracy:",accuracy_score(y_P3, predp53))
print(metrics.confusion_matrix(y_P3, predp53))
print(metrics.classification_report(y_P3, predp53))

AUC: 0.4765984295110548
Accuracy: 0.8889979726386029
[[     3  13905]
 [     2 111376]]
              precision    recall  f1-score   support

           0       0.60      0.00      0.00     13908
           1       0.89      1.00      0.94    111378

   micro avg       0.89      0.89      0.89    125286
   macro avg       0.74      0.50      0.47    125286
weighted avg       0.86      0.89      0.84    125286

